In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

C:\Users\DwaipayanChakroborti\AppData\Roaming\Python\Python310\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
sq = """with allcust as 
(select cust_id from prj-prod-dataplatform.dl_customers_db_raw.tdbk_customer_mtb where cust_id is not null),
  custrejectlist --- List of all customer id where were ever rejected
  AS (
  SELECT
    customerId, min(startApplyDateTime) firstapplicationdate, max(startApplyDateTime) lastapplicationdate, max(applicationStatus) applicationStatus
  FROM
    `risk_credit_mis.loan_master_table`
  WHERE
    applicationStatus LIKE 'REJECT' 
  GROUP BY 
    customerId),
  Activecustlist ---- List of customer id with active loan
  AS (
  SELECT
    customerId,
    digitalLoanAccountId,
    loanPaidStatus
  FROM
    `risk_credit_mis.loan_master_table`
  WHERE
    COALESCE(loanPaidStatus, 'NA') IN ('Normal',
      'In Arrears')),
  RejectcustwithnoActiveLoan --- List of customer id who have a rejected loan earlier and no active current loan
    AS (
    SELECT
      customerId,
      max(applicationStatus) applicationstatus
    FROM
      custrejectlist
    WHERE
      customerId NOT IN (
      SELECT
        customerId
      FROM
        Activecustlist) group by 1),
  RejectcustwithActiveLoan --- List of customer id who have a rejected loan earlier and active current loan
    AS (
    SELECT
      customerId,
      max(applicationStatus) applicationstatus
    FROM
      custrejectlist
    WHERE
      customerId IN (
      SELECT
        customerId
      FROM
        Activecustlist) group by 1),
  TSA_Account as 
  (select distinct customer_id FROM `prj-prod-dataplatform.risk_mart.customer_transactions` WHERE account_type = 'Tonik Account'),
b as 
(select a1.cust_id,
  a2.customerId customerwithrejectloans,
  a3.customerId customerwithactiveloans,
  a4.customerId custwithrejectloanandnotactiveloan,
  a6.customerId custwithrejectloanandactiveloan,
  a5.customer_id Tsaaccountcustomer
  from allcust a1
  left join custrejectlist a2 on a2.customerId = cast(a1.cust_id as numeric)
  left join Activecustlist a3 on a3.customerId = cast(a1.cust_id as numeric)
  left join RejectcustwithnoActiveLoan a4 on a4.customerId = cast(a1.cust_id as numeric)
  left join TSA_Account a5 on a5.customer_id = a1.cust_id
  left join RejectcustwithActiveLoan a6 on a6.customerId = cast(a1.cust_id as numeric)
)
# from b
select * from b
"""

In [3]:
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID c1a13475-a91a-4c86-be81-159f389da1af successfully executed: |          |
Downloading: 100%|██████████|


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470823 entries, 0 to 1470822
Data columns (total 6 columns):
 #   Column                              Non-Null Count    Dtype 
---  ------                              --------------    ----- 
 0   cust_id                             1470823 non-null  object
 1   customerwithrejectloans             264705 non-null   Int64 
 2   customerwithactiveloans             61024 non-null    Int64 
 3   custwithrejectloanandnotactiveloan  264704 non-null   Int64 
 4   custwithrejectloanandactiveloan     1 non-null        Int64 
 5   Tsaaccountcustomer                  341765 non-null   object
dtypes: Int64(4), object(2)
memory usage: 72.9+ MB


In [5]:
print(f"The shape of the dataframe is:\t {df.shape}")
df.head()

The shape of the dataframe is:	 (1470823, 6)


,cust_id,customerwithrejectloans,customerwithactiveloans,custwithrejectloanandnotactiveloan,custwithrejectloanandactiveloan,Tsaaccountcustomer
0,1077512,<NA>,<NA>,<NA>,<NA>,1077512
1,1077404,<NA>,<NA>,<NA>,<NA>,1077404
2,1077519,<NA>,<NA>,<NA>,<NA>,None
3,1077300,<NA>,<NA>,<NA>,<NA>,1077300
4,1077297,1077297,<NA>,1077297,<NA>,1077297


In [6]:
counts = df.count()
counts

cust_id                               1470823
customerwithrejectloans                264705
customerwithactiveloans                 61024
custwithrejectloanandnotactiveloan     264704
custwithrejectloanandactiveloan             1
Tsaaccountcustomer                     341765
dtype: int64

# Customer with TSA Accounts

In [7]:
d = df[(df['cust_id'].notna()) 
       &(df['Tsaaccountcustomer'].notna())].copy()
d.shape

(341765, 6)

In [8]:
d['desc'] = 'Customer with TSA Accounts'
d.head()

,cust_id,customerwithrejectloans,customerwithactiveloans,custwithrejectloanandnotactiveloan,custwithrejectloanandactiveloan,Tsaaccountcustomer,desc
0,1077512,<NA>,<NA>,<NA>,<NA>,1077512,Customer with TSA Accounts
1,1077404,<NA>,<NA>,<NA>,<NA>,1077404,Customer with TSA Accounts
3,1077300,<NA>,<NA>,<NA>,<NA>,1077300,Customer with TSA Accounts
4,1077297,1077297,<NA>,1077297,<NA>,1077297,Customer with TSA Accounts
5,1077386,<NA>,<NA>,<NA>,<NA>,1077386,Customer with TSA Accounts


In [9]:
df1 = d.copy()
df1

,cust_id,customerwithrejectloans,customerwithactiveloans,custwithrejectloanandnotactiveloan,custwithrejectloanandactiveloan,Tsaaccountcustomer,desc
0,1077512,<NA>,<NA>,<NA>,<NA>,1077512,Customer with TSA Accounts
1,1077404,<NA>,<NA>,<NA>,<NA>,1077404,Customer with TSA Accounts
3,1077300,<NA>,<NA>,<NA>,<NA>,1077300,Customer with TSA Accounts
4,1077297,1077297,<NA>,1077297,<NA>,1077297,Customer with TSA Accounts
5,1077386,<NA>,<NA>,<NA>,<NA>,1077386,Customer with TSA Accounts
...,...,...,...,...,...,...,...
1470818,1073701,<NA>,1073701,<NA>,<NA>,1073701,Customer with TSA Accounts
1470819,1073511,<NA>,<NA>,<NA>,<NA>,1073511,Customer with TSA Accounts
1470820,1073619,1073619,<NA>,1073619,<NA>,1073619,Customer with TSA Accounts
1470821,1073608,<NA>,<NA>,<NA>,<NA>,1073608,Customer with TSA Accounts


# Customer with TSA Account but No Active Loan and No Rejected Loan

In [10]:
d = df[(df['cust_id'].notna()) 
       & (df['Tsaaccountcustomer'].notna()) 
       &(df['customerwithactiveloans'].isnull()) 
       &(df['customerwithrejectloans'].isnull())
       & (df['custwithrejectloanandnotactiveloan'].isnull())
       &(df['custwithrejectloanandactiveloan'].isnull())].copy()
d.shape

(250865, 6)

In [11]:
d.head()

,cust_id,customerwithrejectloans,customerwithactiveloans,custwithrejectloanandnotactiveloan,custwithrejectloanandactiveloan,Tsaaccountcustomer
0,1077512,<NA>,<NA>,<NA>,<NA>,1077512
1,1077404,<NA>,<NA>,<NA>,<NA>,1077404
3,1077300,<NA>,<NA>,<NA>,<NA>,1077300
5,1077386,<NA>,<NA>,<NA>,<NA>,1077386
6,1077303,<NA>,<NA>,<NA>,<NA>,1077303


In [12]:
d['desc'] = 'Customer With TSA Account, No Active Loan & No Rejected Loan'
d.head()

,cust_id,customerwithrejectloans,customerwithactiveloans,custwithrejectloanandnotactiveloan,custwithrejectloanandactiveloan,Tsaaccountcustomer,desc
0,1077512,<NA>,<NA>,<NA>,<NA>,1077512,"Customer With TSA Account, No Active Loan & No..."
1,1077404,<NA>,<NA>,<NA>,<NA>,1077404,"Customer With TSA Account, No Active Loan & No..."
3,1077300,<NA>,<NA>,<NA>,<NA>,1077300,"Customer With TSA Account, No Active Loan & No..."
5,1077386,<NA>,<NA>,<NA>,<NA>,1077386,"Customer With TSA Account, No Active Loan & No..."
6,1077303,<NA>,<NA>,<NA>,<NA>,1077303,"Customer With TSA Account, No Active Loan & No..."


In [13]:
df2 = d.copy()
df2

,cust_id,customerwithrejectloans,customerwithactiveloans,custwithrejectloanandnotactiveloan,custwithrejectloanandactiveloan,Tsaaccountcustomer,desc
0,1077512,<NA>,<NA>,<NA>,<NA>,1077512,"Customer With TSA Account, No Active Loan & No..."
1,1077404,<NA>,<NA>,<NA>,<NA>,1077404,"Customer With TSA Account, No Active Loan & No..."
3,1077300,<NA>,<NA>,<NA>,<NA>,1077300,"Customer With TSA Account, No Active Loan & No..."
5,1077386,<NA>,<NA>,<NA>,<NA>,1077386,"Customer With TSA Account, No Active Loan & No..."
6,1077303,<NA>,<NA>,<NA>,<NA>,1077303,"Customer With TSA Account, No Active Loan & No..."
...,...,...,...,...,...,...,...
1470815,1073623,<NA>,<NA>,<NA>,<NA>,1073623,"Customer With TSA Account, No Active Loan & No..."
1470816,1073606,<NA>,<NA>,<NA>,<NA>,1073606,"Customer With TSA Account, No Active Loan & No..."
1470817,1073707,<NA>,<NA>,<NA>,<NA>,1073707,"Customer With TSA Account, No Active Loan & No..."
1470819,1073511,<NA>,<NA>,<NA>,<NA>,1073511,"Customer With TSA Account, No Active Loan & No..."


In [14]:
import pandas as pd
import numpy as np

def fill_empty_columns(main_df, small_df):
    extra_cols = set(small_df.columns) - set(main_df.columns)
    for col in extra_cols:
        main_df[col] = np.nan
    return main_df

df1 = fill_empty_columns(df1, df2)
merged_df = pd.concat([df1, df2], ignore_index=False)
print(f"Merged DataFrame Shape: {merged_df.shape}")
merged_df['desc'].value_counts()

Merged DataFrame Shape: (592630, 7)


desc
Customer with TSA Accounts                                      341765
Customer With TSA Account, No Active Loan & No Rejected Loan    250865
Name: count, dtype: int64

# Customer with TSA Account, with Active Loan and No Rejected Loan

In [15]:
d = df[(df['cust_id'].notna()) 
       & (df['Tsaaccountcustomer'].notna()) 
       &(df['customerwithactiveloans'].notna()) 
       &(df['customerwithrejectloans'].isnull())
       & (df['custwithrejectloanandnotactiveloan'].isnull()) 
       &(df['custwithrejectloanandactiveloan'].isnull())
       ].copy()
d.shape

(55174, 6)

In [16]:
d['desc'] = 'Customer With TSA Account, with Active Loan, no rejected loan'
d.head()

,cust_id,customerwithrejectloans,customerwithactiveloans,custwithrejectloanandnotactiveloan,custwithrejectloanandactiveloan,Tsaaccountcustomer,desc
366,1821659,<NA>,1821659,<NA>,<NA>,1821659,"Customer With TSA Account, with Active Loan, n..."
377,1821628,<NA>,1821628,<NA>,<NA>,1821628,"Customer With TSA Account, with Active Loan, n..."
378,1822136,<NA>,1822136,<NA>,<NA>,1822136,"Customer With TSA Account, with Active Loan, n..."
382,1821399,<NA>,1821399,<NA>,<NA>,1821399,"Customer With TSA Account, with Active Loan, n..."
384,1820725,<NA>,1820725,<NA>,<NA>,1820725,"Customer With TSA Account, with Active Loan, n..."


In [17]:
df3 = d.copy()
len(df3)

55174

In [18]:
import pandas as pd
import numpy as np

def fill_empty_columns(main_df, small_df):
    extra_cols = set(small_df.columns) - set(main_df.columns)
    for col in extra_cols:
        main_df[col] = np.nan
    return main_df

merged_df = pd.concat([df1, df2, df3], ignore_index=False)
print(f"Merged DataFrame Shape: {merged_df.shape}")
merged_df['desc'].value_counts()

Merged DataFrame Shape: (647804, 7)


desc
Customer with TSA Accounts                                       341765
Customer With TSA Account, No Active Loan & No Rejected Loan     250865
Customer With TSA Account, with Active Loan, no rejected loan     55174
Name: count, dtype: int64

# Customer with TSA Account, No Active Loan & has a Rejected Loan

In [19]:
d = df[(df['cust_id'].notna()) 
       & (df['Tsaaccountcustomer'].notna()) 
       &(df['customerwithactiveloans'].isnull()) 
       &(df['customerwithrejectloans'].notna())
       ].copy()
d.shape

(35725, 6)

In [20]:
d['desc'] = 'Customer With TSA Account, No Active Loan, has rejected loan'
d.head()

,cust_id,customerwithrejectloans,customerwithactiveloans,custwithrejectloanandnotactiveloan,custwithrejectloanandactiveloan,Tsaaccountcustomer,desc
4,1077297,1077297,<NA>,1077297,<NA>,1077297,"Customer With TSA Account, No Active Loan, has..."
7,1077484,1077484,<NA>,1077484,<NA>,1077484,"Customer With TSA Account, No Active Loan, has..."
19,1077266,1077266,<NA>,1077266,<NA>,1077266,"Customer With TSA Account, No Active Loan, has..."
170,1077526,1077526,<NA>,1077526,<NA>,1077526,"Customer With TSA Account, No Active Loan, has..."
172,1077279,1077279,<NA>,1077279,<NA>,1077279,"Customer With TSA Account, No Active Loan, has..."


In [21]:
df4 = d.copy()

In [22]:
import pandas as pd
import numpy as np

def fill_empty_columns(main_df, small_df):
    extra_cols = set(small_df.columns) - set(main_df.columns)
    for col in extra_cols:
        main_df[col] = np.nan
    return main_df

merged_df = pd.concat([df1, df2, df3, df4], ignore_index=False)
print(f"Merged DataFrame Shape: {merged_df.shape}")
merged_df['desc'].value_counts()

Merged DataFrame Shape: (683529, 7)


desc
Customer with TSA Accounts                                       341765
Customer With TSA Account, No Active Loan & No Rejected Loan     250865
Customer With TSA Account, with Active Loan, no rejected loan     55174
Customer With TSA Account, No Active Loan, has rejected loan      35725
Name: count, dtype: int64

In [23]:
counts

cust_id                               1470823
customerwithrejectloans                264705
customerwithactiveloans                 61024
custwithrejectloanandnotactiveloan     264704
custwithrejectloanandactiveloan             1
Tsaaccountcustomer                     341765
dtype: int64

# Customer with TSA Account with Active Loan and Rejected Loan

In [24]:
d = df[(df['cust_id'].notna()) 
       & (df['Tsaaccountcustomer'].notna()) 
       &(df['customerwithactiveloans'].notna()) 
       &(df['customerwithrejectloans'].notna())
       &(df['custwithrejectloanandactiveloan'].notna())
       ].copy()
d.shape

(1, 6)

In [25]:
d['desc'] = 'Customer with TSA Account with Active Loan and Rejected Loan'
d.head()

,cust_id,customerwithrejectloans,customerwithactiveloans,custwithrejectloanandnotactiveloan,custwithrejectloanandactiveloan,Tsaaccountcustomer,desc
1104405,1070155,1070155,1070155,<NA>,1070155,1070155,Customer with TSA Account with Active Loan and...


In [26]:
df5 = d.copy()

In [27]:
def fill_empty_columns(main_df, small_df):
    extra_cols = set(small_df.columns) - set(main_df.columns)
    for col in extra_cols:
        main_df[col] = np.nan
    return main_df

merged_df = pd.concat([df1, df2, df3, df4, df5], ignore_index=False)
print(f"Merged DataFrame Shape: {merged_df.shape}")
merged_df['desc'].value_counts()

Merged DataFrame Shape: (683530, 7)


desc
Customer with TSA Accounts                                       341765
Customer With TSA Account, No Active Loan & No Rejected Loan     250865
Customer With TSA Account, with Active Loan, no rejected loan     55174
Customer With TSA Account, No Active Loan, has rejected loan      35725
Customer with TSA Account with Active Loan and Rejected Loan          1
Name: count, dtype: int64

# Customer with No TSA Account but with Rejected Loan

In [28]:
d = df[(df['cust_id'].notna()) 
       & (df['Tsaaccountcustomer'].isnull()) 
       &(df['customerwithactiveloans'].isnull()) 
       &(df['customerwithrejectloans'].notna())
             ].copy()
d.shape

(228979, 6)

In [29]:
d['desc'] = 'Customer with No TSA Account but with Rejected Loan'
d.head()

,cust_id,customerwithrejectloans,customerwithactiveloans,custwithrejectloanandnotactiveloan,custwithrejectloanandactiveloan,Tsaaccountcustomer,desc
107,1077534,1077534,<NA>,1077534,<NA>,None,Customer with No TSA Account but with Rejected...
122,1077508,1077508,<NA>,1077508,<NA>,None,Customer with No TSA Account but with Rejected...
130,1077322,1077322,<NA>,1077322,<NA>,None,Customer with No TSA Account but with Rejected...
131,1077474,1077474,<NA>,1077474,<NA>,None,Customer with No TSA Account but with Rejected...
132,1077529,1077529,<NA>,1077529,<NA>,None,Customer with No TSA Account but with Rejected...


In [30]:
df6 = d.copy()

In [31]:
def fill_empty_columns(main_df, small_df):
    extra_cols = set(small_df.columns) - set(main_df.columns)
    for col in extra_cols:
        main_df[col] = np.nan
    return main_df

merged_df = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=False)
print(f"Merged DataFrame Shape: {merged_df.shape}")
merged_df['desc'].value_counts()

Merged DataFrame Shape: (912509, 7)


desc
Customer with TSA Accounts                                       341765
Customer With TSA Account, No Active Loan & No Rejected Loan     250865
Customer with No TSA Account but with Rejected Loan              228979
Customer With TSA Account, with Active Loan, no rejected loan     55174
Customer With TSA Account, No Active Loan, has rejected loan      35725
Customer with TSA Account with Active Loan and Rejected Loan          1
Name: count, dtype: int64

# Customer with No TSA Account but with Active Loan

In [32]:
d = df[(df['cust_id'].notna()) 
       & (df['Tsaaccountcustomer'].isnull()) 
       &(df['customerwithactiveloans'].notna()) 
       &(df['customerwithrejectloans'].isnull())
             ].copy()
d.shape

(5849, 6)

In [33]:
d['desc']='customer with No TSA Account but with Active Loan'
d.head()

,cust_id,customerwithrejectloans,customerwithactiveloans,custwithrejectloanandnotactiveloan,custwithrejectloanandactiveloan,Tsaaccountcustomer,desc
4779,2415158,<NA>,2415158,<NA>,<NA>,None,customer with No TSA Account but with Active Loan
4782,2415107,<NA>,2415107,<NA>,<NA>,None,customer with No TSA Account but with Active Loan
4783,2415160,<NA>,2415160,<NA>,<NA>,None,customer with No TSA Account but with Active Loan
4833,2415202,<NA>,2415202,<NA>,<NA>,None,customer with No TSA Account but with Active Loan
4869,2415071,<NA>,2415071,<NA>,<NA>,None,customer with No TSA Account but with Active Loan


In [34]:
df7 = d.copy()

In [35]:
def fill_empty_columns(main_df, small_df):
    extra_cols = set(small_df.columns) - set(main_df.columns)
    for col in extra_cols:
        main_df[col] = np.nan
    return main_df

merged_df = pd.concat([df1, df2, df3, df4, df5, df6, df7], ignore_index=False)
print(f"Merged DataFrame Shape: {merged_df.shape}")
merged_df['desc'].value_counts()

Merged DataFrame Shape: (918358, 7)


desc
Customer with TSA Accounts                                       341765
Customer With TSA Account, No Active Loan & No Rejected Loan     250865
Customer with No TSA Account but with Rejected Loan              228979
Customer With TSA Account, with Active Loan, no rejected loan     55174
Customer With TSA Account, No Active Loan, has rejected loan      35725
customer with No TSA Account but with Active Loan                  5849
Customer with TSA Account with Active Loan and Rejected Loan          1
Name: count, dtype: int64

In [36]:
d = df[(df['cust_id'].notna()) 
       & (df['Tsaaccountcustomer'].isnull()) 
       &(df['customerwithactiveloans'].notna()) 
       &(df['customerwithrejectloans'].notna())
             ].copy()
d.shape

(0, 6)

In [37]:
merged_df.to_csv(r"TSA_Account_Description.csv")